In [1]:
import pandas as pd
import numpy as np
import pickle as p

In [2]:
data_all = pd.read_csv('data_all.csv')

In [3]:
np.random.seed(41)
item_subset = list(np.random.choice(data_all['Item_ID'].unique(), size=10, replace=False))
data_subset = data_all.loc[data_all['Item_ID'].isin(item_subset)].copy()

In [4]:
item_subset

[31, 2259, 1203, 5073, 102, 5501, 3818, 4270, 3767, 197]

### `/train`

In [5]:
import requests
import json

url = 'http://localhost:5000/train/'

data = data_subset.to_json()

payload = {'data': data,
           'cv_acc': True,
           'project_id': 1
           }

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

In [6]:
# Model Training Result
r.json()['result']

'Success'

In [7]:
# CV Accuracies Returned
cv_accuracies = pd.DataFrame.from_dict(json.loads(r.json()['cv_acc']))
cv_accuracies

,item_id,avg_sales,r2_score,mae_score,mpe_score,rmse_score
0,31.0,45.218984,0.686220,12.498922,0.276409,17.229308
1,102.0,176.608918,0.772663,33.547006,0.189951,46.804364
2,197.0,55.278581,0.181184,20.998485,0.379867,27.586483
3,1203.0,198.355324,0.778161,31.508546,0.158849,46.504017
4,2259.0,97.155863,0.659568,22.578507,0.232395,30.045301
5,3767.0,286.172376,0.760647,46.156708,0.161290,64.730899
6,3818.0,78.448462,0.692174,15.634456,0.199296,21.413502
7,4270.0,45.090722,-0.162747,23.176727,0.514002,29.737172
8,5073.0,115.369817,0.457243,32.979280,0.285857,42.518965
9,5501.0,464.741615,0.688610,90.295783,0.194292,119.113396


### `/optimize`

In [8]:
url = 'http://localhost:5000/optimize/'

payload = {'project_id': 1,
           'constraints': [],
           'population': 100,
           'max_epoch': 200
           }

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

In [9]:
best_prices = r.json()['result']
best_prices

{'102': 4.21,
 '1203': 13.84,
 '197': 10.96,
 '2259': 4.05,
 '31': 14.51,
 '3767': 9.92,
 '3818': 10.8,
 '4270': 8.9,
 '5073': 4.02,
 '5501': 5.26}

In [10]:
import requests
import json

url = 'http://localhost:5000/predict/'
payload = {'prices': best_prices,
           'project_id': 1
           }
headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }
payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

# What If Prediction
qty = r.json()['qty_estimates']

In [11]:
qty

{'Qty_102': 450,
 'Qty_1203': 619,
 'Qty_197': 117,
 'Qty_2259': 229,
 'Qty_31': 142,
 'Qty_3767': 879,
 'Qty_3818': 223,
 'Qty_4270': 104,
 'Qty_5073': 270,
 'Qty_5501': 954}

In [12]:
# Calculate Revenue
revenue = 0
for item in best_prices.keys():
    revenue += (best_prices[item]*qty['Qty_'+item])
    
print(revenue)

32888.770000000004


### `/predict/`

In [13]:
# Sample Actual Price 
from helper_functions import *

sales_data = data_subset

sales_data = optimize_memory(sales_data)

sales_data_wide = sales_data.set_index(
    ['Wk', 'Tier', 'Store', 'Item_ID']).unstack(level=-1).reset_index().copy()
sales_data_wide.columns = [
    ''.join(str(i) for i in col).strip()
    for col in sales_data_wide.columns.values
]
sales_data_wide = sales_data_wide.sort_values(
    ['Tier', 'Store', 'Wk'], ascending=True).reset_index(drop=True)

sales_data_wide_clean = sales_data_wide.dropna(axis=0).copy()
dataset = sales_data_wide_clean
price_columns = [
    col for col in sales_data_wide_clean.columns if col.startswith('Price')
]

a = dataset.sample(1)
a_dict = a[price_columns].iloc[0].to_dict()
a_input = {}
for key in a_dict.keys():
    a_input[key.split('_')[1]]=round(a_dict[key],2)

print('Input Prices Quantities:')
a_input

Input Prices Quantities:


{'31': 2.6,
 '102': 9.65,
 '197': 10.65,
 '1203': 3.0,
 '2259': 12.7,
 '3767': 10.25,
 '3818': 10.15,
 '4270': 7.85,
 '5073': 3.6,
 '5501': 3.5}

In [14]:
# Predict Sales Qty and Compare to Actual

import requests
import json

url = 'http://localhost:5000/predict/'

payload = {'prices': a_input, 'project_id': 1}

headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)
qty = r.json()['qty_estimates']
qty_outp = {}
for key in qty.keys():
    qty_outp[key] = round(qty[key], 0)
actual = a[[
    col for col in sales_data_wide_clean.columns if col.startswith('Qty_')
]].iloc[0].to_dict()
pd.DataFrame([qty_outp, actual], index=['Estimated', 'Actual']).transpose()

,Estimated,Actual
Qty_102,124.0,143.0
Qty_1203,176.0,243.0
Qty_197,40.0,22.0
Qty_2259,51.0,35.0
Qty_31,35.0,20.0
Qty_3767,293.0,375.0
Qty_3818,49.0,34.0
Qty_4270,29.0,12.0
Qty_5073,110.0,118.0
Qty_5501,368.0,420.0
